This is an example using GulpIO with different python based machine learning libraries. The basic idea here is to illustrate flexibility of GulpIO as a agnostic data loader. It decouples data loading and transforming from network definition hence, you are able to use any DL library of your choice without changing the core data part of your code. 


### Gulp Mnist

Download mnist jpeg version from https://github.com/myleott/mnist_png/blob/master/mnist_png.tar.gz and extract it

In [4]:
! gulp_image_folder --images_per_chunk 5000 --num_workers 8 --shuffle mnist/training/ mnist/trainingGulpIO/
! gulp_image_folder --images_per_chunk 5000 --num_workers 8 --shuffle mnist/testing/ mnist/testingGulpIO/

/bin/sh: 1: gulp_image_folder: not found
/bin/sh: 1: gulp_image_folder: not found


### Define GulpIO Loader

In [2]:
import sys
sys.path.append('/home/egolge/Projects/GulpIO/src/main/python/')

In [17]:
from gulpio.dataset import GulpImageDataset
from gulpio.loader import DataLoader
from gulpio.transforms import Scale, CenterCrop, Compose, UnitNorm

In [39]:
transforms = Compose([
                      Scale(28), 
                      CenterCrop(28),
                      UnitNorm(),
                    ])

In [40]:
dataset = GulpImageDataset('/home/egolge/mnist2/mnist_png/trainingGulpIO/', transform=transforms)
loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0, drop_last=True)

dataset_val = GulpImageDataset('/home/egolge/mnist2/mnist_png/testingGulpIO/', transform=transforms)
loader_val = DataLoader(dataset_val, batch_size=256, shuffle=True, num_workers=0, drop_last=True)

 > Found 12 chunks
 > Found 2 chunks


In [41]:
# Sound check
for data, label in loader:
    print(data.shape)
    break

(256, 28, 28)


### Keras Training

taken from https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

In [87]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical

In [88]:
num_classes = len(dataset.classes)
if K.image_data_format() == 'channels_first':
    input_shape = (1, 32, 32)
else:
    input_shape = (32, 32, 1)

In [89]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [94]:
# train model for 1 epoch
for data, label in loader:
    model.fit(data[:,:,:,None], to_categorical(label, num_classes=num_classes), batch_size=256, epochs=1, verbose=0)
    num_batch += 1


In [98]:
# test model
losses = []
accuracies = []
for data, label in loader_val:
    score = model.evaluate(data[:,:,:,None], to_categorical(label, num_classes=num_classes), verbose=0, batch_size=256)
    losses.append(score[0])
    accuracies.append(score[1])

print('Test loss:', np.mean(losses))
print('Test accuracy:', np.mean(accuracies))

Test loss: 0.0554125563552
Test accuracy: 0.982271634615


### MxNet Training

taken from https://github.com/apache/incubator-mxnet/blob/master/example/svm_mnist/svm_mnist.py

In [7]:
sys.path.append('/home/egolge/mxnet/python/')
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
mx.random.seed(1)

In [21]:
ctx = mx.gpu()
num_fc = 512
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))            
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    # The Flatten layer collapses all axis, except the first one, into one axis.
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(len(dataset.classes)))
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .01})

In [22]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        # pass numpy to mxnet
        data = mx.nd.array(data[:,None,:,:])
        label = mx.nd.array(label)
        
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [23]:
epochs = 10
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(loader):
        # pass numpy to mxnet
        data = mx.nd.array(data[:,None,:,:])
        label = mx.nd.array(label)
        
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0)) 
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)
        
    test_accuracy = evaluate_accuracy(loader_val, net)
    train_accuracy = evaluate_accuracy(loader, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 0.69308744685, Train_acc 0.904263488248, Test_acc 0.909555288462
Epoch 1. Loss: 0.258204011076, Train_acc 0.947282318376, Test_acc 0.949318910256
Epoch 2. Loss: 0.163891134941, Train_acc 0.961121127137, Test_acc 0.965144230769
Epoch 3. Loss: 0.126841373422, Train_acc 0.966045673077, Test_acc 0.968549679487
Epoch 4. Loss: 0.105358349017, Train_acc 0.972472622863, Test_acc 0.974959935897
Epoch 5. Loss: 0.0904899819418, Train_acc 0.975143563034, Test_acc 0.978365384615
Epoch 6. Loss: 0.0816053879402, Train_acc 0.977614182692, Test_acc 0.979266826923
Epoch 7. Loss: 0.0745786741858, Train_acc 0.980201655983, Test_acc 0.981270032051
Epoch 8. Loss: 0.0673958410738, Train_acc 0.976462339744, Test_acc 0.976862980769
Epoch 9. Loss: 0.0640447250246, Train_acc 0.983072916667, Test_acc 0.983573717949


### PyTorch

In [42]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [46]:
epochs = 10
log_interval = 10

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()
model.cuda()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = torch.FloatTensor(data[:, None, :, :]).cuda(), torch.LongTensor(target).cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in loader_val:
        data, target = torch.FloatTensor(data[:, None, :, :]).cuda(), torch.LongTensor(target).cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(loader_val.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(loader_val.dataset),
        100. * correct / len(loader_val.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.334383
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.289639
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.255640
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.217098
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.102788
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.814367
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.286950
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.140132
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.022240
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.880636
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.802292
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.801508
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.659856
Train Epoch: 1 [33280/60000 (56%)]	Loss: 0.695324
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.787371
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.581242
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.507682
Train Epoch: 1 [43520/60000 (73%)]	Loss: 0.705761
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.442291
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.508487
Train Epoc

KeyboardInterrupt: 